## project_nutrients_review_negative

In [1]:
import numpy as np

In [2]:
# project_nutrients_review_negative.csv
import pandas as pd
df_review_negative = pd.read_csv('../datasets/project_nutrients_review_negative.csv')

In [ ]:
df_review_negative.info()

In [4]:
df_review_negative = df_review_negative.dropna()

### LDA 분석

##### 문장 벡터화

In [5]:
from gensim.corpora import Dictionary

In [ ]:
# df_review_negative dictionary
dictionary = Dictionary(df_review_negative[['replaced_review']].values)
dictionary

In [ ]:
dictionary.doc2bow(df_review_negative[['replaced_review']].values[3])

In [8]:
# dictionary에 의한 한글 문장을 벡터화 변환
corpus_list = list()
for sentence in df_review_negative[['replaced_review']].values :
  vectors = dictionary.doc2bow(sentence)
  corpus_list.append(vectors)

In [9]:
df_review_negative['문장벡터화'] = corpus_list

In [ ]:
df_review_negative.columns

##### 토픽 잡기
- review_negative.csv 토픽 분석
- 토픽 수 9

In [11]:
from gensim.models import LdaModel

In [12]:
lda_model = LdaModel(corpus=corpus_list, id2word=dictionary, num_topics=9)

In [13]:
lda_model.print_topics(num_words=4)

[(0,
  '0.156*"쭈욱 제품 11 절때 할인 주문 상세 상세" + 0.156*"박스 왕창 파손 찢어지다 배송 돼다 기분 좋다" + 0.016*"배송 빨르다 성능 성능 별로 유통 기한 기한 넉넉하다 박스 규칙 혈압 측정 복용 전과 복용 개월차 별다르다 개선" + 0.016*"배송 느리다 성능 성능 뛰어나다 유통 기한 기한 넉넉하다 배송 느리다"'),
 (1,
  '0.182*"택배 박스 젖다 제품 괜찬을줄 안다 약품 아니다 건강 식품 관리 보관 상태 중요 제품 박스 제품 젖다 괜찬을줄알 박스 개봉 해보다 박스 눌렸는 알약 튀어 벗기다 정말 제품 싸다 가격 아니다 다시 반품" + 0.123*"대웅제약 우루사 알려지다 믿다 구매 판매 제조 코스 맥스 바이오 우쨋 먹기 재다 매다 병원 건강 안좋다 신경 써다" + 0.123*"배송 느리다 성능 성능 뛰어나다 유통 기한 기한 넉넉하다 배송 느리다" + 0.123*"줄줄"'),
 (2,
  '0.206*"배송 빨르다 성능 성능 별로 유통 기한 기한 넉넉하다 박스 규칙 혈압 측정 복용 전과 복용 개월차 별다르다 개선" + 0.140*"0원 구매 쿄릭홍국 제품 품절 나서다 구매 몰리다 갑자기 가격 만원 올리다 판매 저렴하다 링크 남다 구매 하렵니" + 0.140*"아쉽다 안쪽 남편 복용 먹기" + 0.073*"냄새 냄새 궁금하다 찝찝하다 속도 안좋다 아프다"'),
 (3,
  '0.174*"배송 느리다 성능 성능 뛰어나다 유통 기한 기한 넉넉하다 배송 느리다" + 0.092*"가격 만원" + 0.092*"검사 결과 크다 변화" + 0.092*"배송 성능 성능 별로 유통 기한 기한 짧다"'),
 (4,
  '0.110*"주문 해외 구매 대행 제품 반품 안되다 연락 안되다 차다 어이없다" + 0.058*"먹기 혈액 검사 비교 해보다 레스 트롤 수치 감소 감마 수치 상승 야하다 양제 정확하다 상관관계 알다 의심 상황 복용 쉬다 기간 야하다" + 0.058*"배송 빨르다 성능 성능 별로 유통 기한 기한 넉넉하다 박스 규칙 

##### 최적에 토픽 단어 수

In [ ]:
sentences = df_review_negative['replaced_review']
sentences

In [ ]:
type(sentences)

In [ ]:
# 각 문장을 공백으로 나누어 리스트에 저장
tokenized_sentences = [sentence.split() for sentence in sentences]

In [16]:
# 결과 출력
morphs_list = list()
for tokens in tokenized_sentences:
    morphs_list.append(tokens)
morphs_list

In [20]:
preprocessed_sentences = morphs_list

In [ ]:
dictionary = Dictionary(preprocessed_sentences) # fix in like sklean
dictionary

In [ ]:
# 일관성 점수
from gensim.models.coherencemodel import CoherenceModel
coherenceModel = CoherenceModel(model=lda_model, texts=morphs_list, dictionary=dictionary)
coherenceModel.get_coherence()

In [ ]:
# 혼잡도 점수
lda_model.log_perplexity(corpus_list)

In [24]:
start_topic = 2
end_topic = 10
coherence_scores = list()
perplexity_scores = list()
for topic_number in range(start_topic, end_topic+1):
    lda_model = LdaModel(corpus=corpus_list, id2word=dictionary, num_topics=topic_number) # fix
    coherenceModel = CoherenceModel(model=lda_model, texts=morphs_list, dictionary=dictionary)
    coherence_scores.append(coherenceModel.get_coherence()) # 일관성 점수
    perplexity_scores.append(lda_model.log_perplexity(corpus_list))

In [ ]:
import pandas as pd
scores = pd.DataFrame([coherence_scores,  perplexity_scores]).T
scores

In [26]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.lineplot(coherence_scores)
plt.show()
sns.lineplot(perplexity_scores)
plt.show()

##### LDA 시각화(review_negative)

In [28]:
# %pip install pyLDAvis

In [29]:
import pyLDAvis.gensim_models

In [30]:
pyLDAvis.enable_notebook() # 일반적인 python에선 불필요
result_visualized = pyLDAvis.gensim_models.prepare(lda_model, corpus_list, dictionary)

In [31]:
pyLDAvis.display(result_visualized)

In [ ]:
# project_nutrients_review_negative_result_visualized.html
pyLDAvis.save_html(result_visualized, '../datasets/project_nutrients_review_negative_result_visualized.html')

## project_nutrients_review_positive

In [ ]:
import pandas as pd
df_review_positive = pd.read_csv('../datasets/project_nutrients_review_positive.csv')

In [33]:
df_review_positive.info()

In [35]:
df_review_positive = df_review_positive.dropna()

### LDA 분석

##### 문장 벡터화

In [36]:
# 'replaced_review' 열의 모든 값을 문자열로 변환
df_review_positive['replaced_review'] = df_review_positive['replaced_review'].astype(str)

In [ ]:
dictionary = Dictionary(df_review_positive[['replaced_review']].values)
dictionary

In [38]:
# dictionary에 의한 한글 문장을 벡터화 변환
corpus_list = list()
for sentence in df_review_positive[['replaced_review']].values :
  vectors = dictionary.doc2bow(sentence)
  corpus_list.append(vectors)

In [39]:
df_review_positive['문장벡터화'] = corpus_list

In [ ]:
df_review_positive[:3]

##### 토픽 잡기(review_positive)
- review_positive.csv 토픽 분석
- 토픽 수 9 <<<<<<< 변경하기 

In [41]:
lda_model = LdaModel(corpus=corpus_list, id2word=dictionary, num_topics=9)

In [ ]:
lda_model.print_topics(num_words=4)

##### 최적에 토픽 단어 수

In [ ]:
sentences = df_review_positive['replaced_review']
sentences

In [ ]:
type(sentences)

In [ ]:
# 각 문장을 공백으로 나누어 리스트에 저장
tokenized_sentences = [sentence.split() for sentence in sentences]

In [ ]:
# 결과 출력
morphs_list = list()
for tokens in tokenized_sentences:
    morphs_list.append(tokens)
morphs_list

In [ ]:
preprocessed_sentences = morphs_list

In [ ]:
dictionary = Dictionary(preprocessed_sentences) # fix in like sklean
dictionary

In [ ]:
# 일관성 점수
from gensim.models.coherencemodel import CoherenceModel
coherenceModel = CoherenceModel(model=lda_model, texts=morphs_list, dictionary=dictionary)
coherenceModel.get_coherence()

In [ ]:
# 혼잡도 점수
lda_model.log_perplexity(corpus_list)

In [ ]:
start_topic = 2
end_topic = 10
coherence_scores = list()
perplexity_scores = list()
for topic_number in range(start_topic, end_topic+1):
    lda_model = LdaModel(corpus=corpus_list, id2word=dictionary, num_topics=topic_number) # fix
    coherenceModel = CoherenceModel(model=lda_model, texts=morphs_list, dictionary=dictionary)
    coherence_scores.append(coherenceModel.get_coherence()) # 일관성 점수
    perplexity_scores.append(lda_model.log_perplexity(corpus_list))

In [ ]:
import pandas as pd
scores = pd.DataFrame([coherence_scores,  perplexity_scores]).T
scores

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sns.lineplot(coherence_scores)
plt.show()
sns.lineplot(perplexity_scores)
plt.show()

##### LDA 시각화

In [43]:
pyLDAvis.enable_notebook() # 일반적인 python에선 불필요
result_visualized = pyLDAvis.gensim_models.prepare(lda_model, corpus_list, dictionary)

In [44]:
pyLDAvis.display(result_visualized)

In [45]:
pyLDAvis.save_html(result_visualized, '../datasets/project_nutrients_review_positive_result_visualized.html')